In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [24]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS2_3classes__Goertzel.csv'
dataset_DS2 = pd.read_csv(os.path.join(rootDir, path, filename));

filename = 'MIT-BIH__DS1_3classes__Goertzel.csv'
dataset_DS1 = pd.read_csv(os.path.join(rootDir, path, filename));

In [25]:
dataset_DS1.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33
0,0.151058,0.128256,0.099898,0.083728,0.068214,0.046109,0.026629,0.017688,0.011045,0.009385,...,0.000768,0.001601,0.000325,0.001421,0.001017,0.001651,0.001060,0.000862,0.000547,0
1,0.147046,0.124114,0.094705,0.077863,0.059560,0.042234,0.025960,0.013447,0.011329,0.006718,...,0.000888,0.000654,0.000932,0.001443,0.000800,0.000732,0.000521,0.000157,0.000781,0
2,0.168356,0.139827,0.106008,0.084504,0.063497,0.040942,0.019961,0.012706,0.006084,0.010158,...,0.001229,0.001054,0.000990,0.000836,0.001372,0.000819,0.000618,0.001440,0.001406,0
3,0.175502,0.144138,0.109498,0.088609,0.062034,0.037186,0.022075,0.012085,0.008090,0.008089,...,0.000907,0.000763,0.001116,0.001679,0.000606,0.001112,0.000735,0.001248,0.000391,0
4,0.177210,0.147767,0.114982,0.092439,0.064739,0.038087,0.020698,0.013635,0.007597,0.008840,...,0.001015,0.000333,0.000903,0.001165,0.000636,0.001016,0.000668,0.001149,0.000859,0


In [26]:
dataset_DS1.shape

(50969, 33)

# Data preprocessing

In [27]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

In this step we are going standardize our dataset. The pipeline.pkl will be loaded, and it contains the weights of normalization onf DS1 set.

In [28]:
data_std_DS1 = std_scaler.fit_transform(dataset_DS1.values[:,:-1])
data_label_DS1 = dataset_DS1.values[:,-1]

data_std_DS2 = std_scaler.transform(dataset_DS2.values[:,:-1])
data_label_DS2 = dataset_DS2.values[:,-1]

# Load classifiers

In [29]:
random_search = joblib.load('./Models/classifiers.pkl')

In [30]:
from sklearn.externals import joblib

random_search = joblib.load('./Models/classifiers.pkl')
mlp_clf = random_search['MLP'].best_estimator_
svm_clf = random_search['SVM-RBF'].best_estimator_

from sklearn.naive_bayes import GaussianNB

bayes_clf = GaussianNB()

classifiers = {'MLP-238': mlp_clf, 'SVM-RBF': svm_clf, 'Naive-Bayes': bayes_clf}

# TRAIN on DS1 and **TEST** on DS2 set

In [31]:
clf_outputs = {'true': dict((k, {}) for k in classifiers.keys()), 
               'pred': dict((k, {}) for k in classifiers.keys())}
results = dict((k, {}) for k in classifiers.keys())

for clf in classifiers.keys():
    classifiers[clf].fit(data_std_DS1, data_label_DS1)
        
    clf_outputs['true'][clf][0] = data_label_DS2
    clf_outputs['pred'][clf][0] = classifiers[clf].predict(data_std_DS2)
    
    print(clf) 

MLP-238
SVM-RBF
Naive-Bayes


# Save results to CSVs and figures

In [32]:
clf_outputs

{'true': {'MLP-238': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([0., 0., 0., ..., 0., 0., 0.])}},
 'pred': {'MLP-238': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'SVM-RBF': {0: array([0., 0., 0., ..., 0., 0., 0.])},
  'Naive-Bayes': {0: array([1., 1., 1., ..., 0., 0., 1.])}}}

In [33]:
results = results_clf(3, clf_outputs['true'], clf_outputs['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Goertzel/Experiment_07_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:150: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,2] = TP / (TP + FP)


In [34]:
export_results(results, 'validation')

MLP-238
SVM-RBF
Naive-Bayes


/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Goertzel/Experiment_07_scikitLearn__allCLF_onDS2_allFeatures_3Classes/experiment.py:286: RuntimeWarning: invalid value encountered in true_divide
  return np.around((confMat / np.sum(confMat,axis=1)[:,None])*100,2)


In [35]:
save_models(classifiers)
save_pipeline(std_scaler)